In [ ]:
import mlflow
from mlflow.tracking import MlflowClient
import os
import time
import sys

print("=" * 70)
print("PRODUCTION MODEL PROMOTION")
print("=" * 70)

# =============================================================================
# CONFIGURATION
# =============================================================================
UC_CATALOG_NAME = "workspace"
UC_SCHEMA_NAME = "ml"
MODEL_NAME = f"{UC_CATALOG_NAME}.{UC_SCHEMA_NAME}.house_price_model_uc"

STAGING_ALIAS = "staging"
PRODUCTION_ALIAS = "production"

# Quality thresholds for production promotion
MIN_R2_FOR_PROD = 0.80
MAX_MAPE_FOR_PROD = 10.0

# =============================================================================
# MLflow Setup
# =============================================================================
try:
    if "DATABRICKS_RUNTIME_VERSION" in os.environ:
        mlflow.set_registry_uri("databricks-uc")
        print("MLflow configured for Unity Catalog")
    
    client = MlflowClient()
    print("MLflow client initialized")
except Exception as e:
    print(f"Error initializing MLflow: {e}")
    sys.exit(1)

# =============================================================================
# VALIDATE STAGING MODEL BEFORE PROMOTION
# =============================================================================
print(f"\nValidating Staging model before promotion...")
print(f"Model: {MODEL_NAME}")

try:
    # Get all model versions
    model_versions = client.search_model_versions(f"name='{MODEL_NAME}'")
    
    if not model_versions:
        print(f"Error: No versions found for model {MODEL_NAME}")
        sys.exit(1)
    
    # Find version with Staging alias
    # FIX: Must use get_model_version() to get aliases in Unity Catalog
    staging_version = None
    for version in model_versions:
        # Get full version details with aliases
        full_version = client.get_model_version(MODEL_NAME, version.version)
        
        if STAGING_ALIAS in full_version.aliases:
            staging_version = full_version
            break
    
    if not staging_version:
        print(f"Error: No model version found with '{STAGING_ALIAS}' alias")
        print("Please run UAT pipeline first to set Staging alias")
        sys.exit(1)
    
    print(f"\nFound Staging Model:")
    print(f"  Version: {staging_version.version}")
    print(f"  Status: {staging_version.status}")
    print(f"  Run ID: {staging_version.run_id}")
    print(f"  Aliases: {', '.join(staging_version.aliases)}")
    
    # Get model metrics from run
    run = client.get_run(staging_version.run_id)
    
    r2_score = run.data.metrics.get('r2_score', 0)
    rmse = run.data.metrics.get('rmse', float('inf'))
    
    print(f"\nModel Metrics:")
    print(f"  R² Score: {r2_score:.4f}")
    print(f"  RMSE: ${rmse:,.2f}")
    
    # Quality checks
    print(f"\nProduction Quality Checks:")
    
    quality_checks = []
    
    if r2_score >= MIN_R2_FOR_PROD:
        quality_checks.append(f"  PASS: R² {r2_score:.4f} >= {MIN_R2_FOR_PROD}")
    else:
        quality_checks.append(f"  FAIL: R² {r2_score:.4f} < {MIN_R2_FOR_PROD}")
    
    # Print results
    for check in quality_checks:
        print(check)
    
    # Check if all passed
    if not all("PASS" in check for check in quality_checks):
        print(f"\nPromotion DENIED: Model does not meet production quality standards")
        sys.exit(1)
    
    print(f"\nQuality validation PASSED - Model eligible for production")
    
except Exception as e:
    print(f"Error during validation: {e}")
    import traceback
    traceback.print_exc()
    sys.exit(1)

# =============================================================================
# WAIT FOR MODEL TO BE READY
# =============================================================================
print(f"\nChecking model readiness...")

max_wait_time = 300
elapsed_time = 0
check_interval = 5

while elapsed_time < max_wait_time:
    current_version = client.get_model_version(MODEL_NAME, staging_version.version)
    status = current_version.status
    
    if status == "READY":
        print(f"Model version {staging_version.version} is READY")
        break
    elif status == "FAILED_REGISTRATION":
        print(f"Error: Model registration failed")
        sys.exit(1)
    else:
        print(f"  Status: {status} - waiting {check_interval}s...")
        time.sleep(check_interval)
        elapsed_time += check_interval

if elapsed_time >= max_wait_time:
    print(f"Timeout: Model not ready after {max_wait_time}s")
    sys.exit(1)

# =============================================================================
# PROMOTE TO PRODUCTION
# =============================================================================
print(f"\n{'=' * 70}")
print("PROMOTING MODEL TO PRODUCTION")
print(f"{'=' * 70}")

try:
    # Check if Production alias already exists on another version
    # FIX: Must use get_model_version() for each version to check aliases
    prod_version_exists = False
    for version in model_versions:
        full_version = client.get_model_version(MODEL_NAME, version.version)
        
        if PRODUCTION_ALIAS in full_version.aliases and full_version.version != staging_version.version:
            print(f"\nWarning: Production alias currently on version {full_version.version}")
            print(f"Will move alias to version {staging_version.version}")
            prod_version_exists = True
            break
    
    # Set Production alias on staging version
    print(f"\nSetting '{PRODUCTION_ALIAS}' alias on version {staging_version.version}...")
    
    client.set_registered_model_alias(
        MODEL_NAME, 
        PRODUCTION_ALIAS, 
        staging_version.version
    )
    
    print(f"Production alias set successfully")
    
    # Verify the alias was set
    time.sleep(2)  # Brief wait for consistency
    
    updated_version = client.get_model_version(MODEL_NAME, staging_version.version)
    
    if PRODUCTION_ALIAS in updated_version.aliases:
        print(f"\n{'=' * 70}")
        print("PROMOTION SUCCESSFUL")
        print(f"{'=' * 70}")
        print(f"Model Version: {staging_version.version}")
        print(f"Aliases: {', '.join(updated_version.aliases)}")
        print(f"Status: PRODUCTION READY")
        print(f"\nModel URI: models:/{MODEL_NAME}@{PRODUCTION_ALIAS}")
        print(f"{'=' * 70}")
    else:
        print(f"Warning: Alias may not have been set properly")
        print(f"Please verify in MLflow UI")
    
    # Log promotion metadata
    print(f"\nPromotion Metadata:")
    print(f"  Promoted By: Automated Pipeline")
    print(f"  Promotion Time: {time.strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"  Source Alias: {STAGING_ALIAS}")
    print(f"  Target Alias: {PRODUCTION_ALIAS}")
    print(f"  Model Quality: R²={r2_score:.4f}, RMSE=${rmse:,.2f}")
    
except Exception as e:
    print(f"\nError during promotion: {e}")
    import traceback
    traceback.print_exc()
    sys.exit(1)

# =============================================================================
# FINAL MESSAGE
# =============================================================================
print(f"\n{'=' * 70}")
print("NEXT STEPS:")
print(f"{'=' * 70}")
print("1. Create/Update Serving Endpoint with Production model")
print("2. Run production inference tests")
print("3. Monitor model performance in production")
print("4. Set up alerts for model drift")
print(f"{'=' * 70}")

# Success exit for pipeline
try:
    dbutils.notebook.exit("PROMOTION_SUCCESS")
except:
    pass